In [88]:
import numpy as np
jobs = [[(0, 7), (1, 5), (2, 2), (3, 3), (4, 9)],
       [(0, 6), (1, 6), (2, 4), (3, 5), (4, 10)],
       [(0, 5), (1, 4), (2, 5), (3, 6), (4, 8)],
       [(0, 8), (1, 3), (2, 3), (3, 2), (4, 6)]]
n = len(jobs)
m = len(jobs[0])


In [89]:
def translate_to_matrix(jobs_data):
    translated_matrix = np.zeros((n,m))
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            translated_matrix[job_id][task_id] = task[1]
    return translated_matrix

def check_conditions(translated_matrix): 
    mins = np.amin(translated_matrix, axis=0)
    maxs = np.amax(translated_matrix, axis=0)
    one_to_m_minus_one_maxs = np.amax(maxs[1:m-1], axis=0)
    if (mins[0] >= one_to_m_minus_one_maxs):
        return 'first'
    if (mins[m - 1] >= one_to_m_minus_one_maxs):
        return 'second'
    return False


In [90]:
translated_matrix = translate_to_matrix(jobs)
print(check_conditions(translated_matrix))
xi = np.zeros(n, dtype=np.int)
yi = np.zeros(n, dtype=np.int)
for job_id, job in enumerate(translated_matrix):
    xi[job_id] = np.sum(job[0:len(job)-1])
    yi[job_id] = np.sum(job[1:len(job)])
    
print (xi, yi)

def add_shifting(shift, elem_ind):
    for shift_ind, shifting in enumerate(shift):
        if (shift_ind >= elem_ind):
            shift[shift_ind]+=1

sequence = np.zeros(n, dtype=np.int)
shifting = np.zeros(n, dtype=np.int)
start_ind = 0
end_ind = n - 1
for k in range(n):
    x_min = np.amin(xi, axis=0)
    x_min_ind = np.argmin(xi)
    y_min = np.amin(yi, axis=0)
    y_min_ind = np.argmin(yi)
    if(x_min < y_min):
        sequence[start_ind] = x_min_ind + shifting[x_min_ind]
        add_shifting(shifting, x_min_ind)
        xi = np.delete(xi, x_min_ind)
        yi = np.delete(yi, x_min_ind)
        start_ind+=1
    else:
        sequence[end_ind] = y_min_ind + shifting[y_min_ind]
        add_shifting(shifting, y_min_ind)
        xi = np.delete(xi, y_min_ind)
        yi = np.delete(yi, y_min_ind)
        end_ind-=1
print (sequence)

second
[17 21 20 16] [19 25 23 14]
[0 2 1 3]


In [91]:
import collections
from ortools.sat.python import cp_model

In [92]:
jobs_data = [[(0, 7), (1, 5), (2, 2), (3, 3), (4, 9)],
       [(0, 6), (1, 6), (2, 4), (3, 5), (4, 10)],
       [(0, 5), (1, 4), (2, 5), (3, 6), (4, 8)],
       [(0, 8), (1, 3), (2, 3), (3, 2), (4, 6)]]

machines_count = 1 + max(task[0] for job in jobs_data for task in job)
all_machines = range(machines_count)

horizon = sum(task[1] for job in jobs_data for task in job)
model = cp_model.CpModel()
all_tasks = {}
machine_to_intervals = collections.defaultdict(list)

task_type = collections.namedtuple('task_type', 'start end interval')
assigned_task_type = collections.namedtuple('assigned_task_type', 'start job index duration')

for job_id, job in enumerate(jobs_data):
    for task_id, task in enumerate(job):
        machine = task[0]
        duration = task[1]
        suffix = '_%i_%i' % (job_id, task_id)
        start_var = model.NewIntVar(0, horizon, 'start' + suffix)
        end_var = model.NewIntVar(0, horizon, 'end' + suffix)
        interval_var = model.NewIntervalVar(start_var, duration, end_var, 'interval' + suffix)
        all_tasks[job_id, task_id] = task_type(
            start=start_var, end=end_var, interval=interval_var)
        machine_to_intervals[machine].append(interval_var)

for machine in all_machines:
    model.AddNoOverlap(machine_to_intervals[machine])

for job_id, job in enumerate(jobs_data):
    for task_id in range(len(job) - 1):
        model.Add(all_tasks[job_id, task_id + 1].start >= all_tasks[job_id, task_id].end)

obj_var = model.NewIntVar(0, horizon, 'makespan')

model.AddMaxEquality(obj_var, [
    all_tasks[job_id, len(job) - 1].end
    for job_id, job in enumerate(jobs_data)
])
model.Minimize(obj_var)

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    assigned_jobs = collections.defaultdict(list)
    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            assigned_jobs[machine].append(
                assigned_task_type(
                    start=solver.Value(all_tasks[job_id, task_id].start),
                    job=job_id,
                    index=task_id,
                    duration=task[1]))
    output = ''
    for machine in all_machines:
        assigned_jobs[machine].sort()
        sol_line_tasks = 'Машина ' + str(machine) + ': '
        sol_line = '          '
        
        for assigned_task in assigned_jobs[machine]:
            name = 'подзадача_%i_%i' % (assigned_task.job, assigned_task.index)
            sol_line_tasks += '%-20s' % name
            start = assigned_task.start
            duration = assigned_task.duration
            sol_tmp = '[%i,%i]' % (start, start + duration)
            sol_line += '%-20s' % sol_tmp
            
        sol_line += '\n'
        sol_line_tasks += '\n'
        output += sol_line_tasks
        output += sol_line

    print('Время оптимального решения: %i' % solver.ObjectiveValue())
    print('\n')
    print(output)

Время оптимального решения: 51


Машина 0: подзадача_0_0       подзадача_2_0       подзадача_1_0       подзадача_3_0       
          [0,7]               [7,12]              [12,18]             [18,26]             
Машина 1: подзадача_0_1       подзадача_2_1       подзадача_1_1       подзадача_3_1       
          [7,12]              [12,16]             [18,24]             [26,29]             
Машина 2: подзадача_0_2       подзадача_2_2       подзадача_1_2       подзадача_3_2       
          [12,14]             [16,21]             [24,28]             [29,32]             
Машина 3: подзадача_0_3       подзадача_2_3       подзадача_1_3       подзадача_3_3       
          [14,17]             [21,27]             [28,33]             [33,35]             
Машина 4: подзадача_0_4       подзадача_2_4       подзадача_1_4       подзадача_3_4       
          [17,26]             [27,35]             [35,45]             [45,51]             

